# Interactive Visualization
## CMU Data Science Club
### Husni Almoubayyed

Follow along by visiting https://bit.ly/DSCInteractiveViz

Note: Interactive plots in this notebook will not appear in the GitHub view, You can view this including the plots in the html version [here](http://htmlpreview.github.io/?https://github.com/hsnee/CMU-DSC-Talks/blob/master/Data_Visualization/InteractiveVisualization.html), or you can download this notebook and run it locally to create the plots.

In [ ]:
# pip install packages we're gonna use
!pip install --upgrade numpy plotly pandas pandas_datareader 

In this tutorial we will explore interactive data visualization with Plotly. First, you need to make an account at https://plot.ly/Auth/login/?action=signup#/, and then go to https://plot.ly/settings/api, click regenerate key, copy it, and paste it in the `api_key` argument in the cell below, along with your username

In [ ]:
import plotly
plotly.tools.set_credentials_file(username='', api_key='')

In [1]:
# import libraries we're gonna use
import numpy as np
import pandas_datareader.data as web
import datetime
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go

We talked about `pandas` last time. `pandas` actually stands for 'panel data' and was created with finance in mind (woth some core developers working at hedge funds). It provides an API to fetch financial data from Yahoo Finance, Morningstar, etc.

We're gonna look at financial data (stock market prices) fetched from Yahoo Finance using pandas_datareader. `Datetime` is a nice python library that handles dates and times consistently in a way that many other libraries, such as pandas, use. Let's fetch data from the start of 2008 to the start of 2018.

In [2]:
start = datetime.datetime(2008, 1, 1)
end = datetime.datetime(2018, 1, 1)

In [3]:
# This fetches the stock prices for the S%P 500 for the dates we selected from Yahoo Finance.
spy_df = web.DataReader('SPY', 'yahoo', start, end).reset_index()

In [4]:
spy_df.head()

,Date,High,Low,Open,Close,Volume,Adj Close
0,2008-01-02,146.990005,143.880005,146.529999,144.929993,204935600.0,116.068459
1,2008-01-03,145.490005,144.070007,144.910004,144.860001,125133300.0,116.012428
2,2008-01-04,143.440002,140.910004,143.339996,141.309998,232330900.0,113.169357
3,2008-01-07,142.229996,140.100006,141.809998,141.190002,234991000.0,113.073250
4,2008-01-08,142.899994,138.440002,142.080002,138.910004,326365700.0,111.247330


In [4]:
# Our first interaction with Plotly. Plotly can create tables when passed a pandas DataFrame
py.iplot(ff.create_table(spy_df.head()))

That wasn't extremely helpful... Let's try something more interesting.

In general, the `plotly` syntax goes like this:

`data`: a list of `plotly.graph_objs`, or a list of dictionaries.

`layout`: usually optional, controls the aesthetics of your plot

then you call

`py.iplot(data, layout)`

Let's look at some examples:

In [5]:
data = [go.Scatter(x=spy_df.Date, y=spy_df.Close)]

py.iplot(data)

In [6]:
data = [go.Candlestick(x=spy_df.Date[0:90],
                       open=spy_df.Open[0:90],
                       high=spy_df.High[0:90],
                       low=spy_df.Low[0:90],
                       close=spy_df.Close[0:90])]
py.iplot(data)

Let's define a function `rollingReturns` that returns the 1-year returns for an invesment in a stock:

In [8]:
def rollingReturns(df, lookback=360):
    x = df.Date
    y = df.Close
    return [y[i]/y[i-lookback] for i in range(lookback,len(list(y)))]

In [9]:
data = [go.Scatter(x=spy_df.Date, y=rollingReturns(spy_df))]

py.iplot(data)

## Adding buttons:

In [10]:
aapl_df = web.DataReader('AAPL', 'yahoo', start, end).reset_index()
data = [go.Scatter(x=spy_df.Date, y=spy_df.Close, name='S%P 500'),
        go.Scatter(x=aapl_df.Date, y=aapl_df.Close, name='AAPL') ]


# updatemenus tells plotly what to show and what to hide when a button is pressed
updatemenus = list([
    dict(active = -1, # initialize with no buttons active
         buttons = list([ # buttons is a list of dictionaries 
            dict(label = 'SPY',
                 method = 'update',
                 args = [{'visible': [True, False]},
                         {'title': 'S&P 500'}]),
            dict(label = 'AAPL',
                 method = 'update',
                 args = [{'visible': [False, True]},
                         {'title': 'Apple'}]),
            dict(label = 'Both',
                 method = 'update',
                 args = [{'visible': [True, True]},
                         {'title': 'Apple vs S&P 500'}])
        ]),
    )
])

# layout here is not optional if you would like the buttons to show up
layout = dict(title='Yahoo Finance', showlegend=True,
              updatemenus=updatemenus)

fig = dict(data=data, layout=layout)
f = py.iplot(fig, filename='update_dropdown')
f

In [11]:
aapl_df = web.DataReader('AAPL', 'yahoo', start, end).reset_index()
data = [go.Scatter(x=spy_df.Date, y=rollingReturns(spy_df), name='S&P 500'),
        go.Scatter(x=aapl_df.Date, y=rollingReturns(aapl_df), name='AAPL')]


# updatemenus tells plotly what to show and what to hide when a button is pressed
updatemenus = list([
    dict(active = -1, # initialize with no buttons active
         buttons = list([ # buttons is a list of dictionaries 
            dict(label = 'SPY',
                 method = 'update',
                 args = [{'visible': [True, False]},
                         {'title': 'S&P 500'}]),
            dict(label = 'AAPL',
                 method = 'update',
                 args = [{'visible': [False, True]},
                         {'title': 'Apple'}]),
            dict(label = 'Both',
                 method = 'update',
                 args = [{'visible': [True, True]},
                         {'title': 'Apple vs S&P 500'}])
        ]),
    )
])

# layout here is not optional if you would like the buttons to show up
layout = dict(title='Yahoo Finance', showlegend=True,
              updatemenus=updatemenus)

fig = dict(data=data, layout=layout)
f = py.iplot(fig, filename='update_dropdown')
f

You can also view your plots online by going to your account, or pressing the `Edit Chart` button in a plot
